# 0. Install Pytorch
<hr>

In [46]:
!pip install torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


# 1. Setting
<hr>

import library

In [49]:
# import torch
import torch
import torch.nn as nn
torch.__version__

# using gpu 
use_cuda = True
device = 'cpu'
if use_cuda and torch.cuda.is_available():
    net.cuda()
    device = 'cuda'
  
print(device)

cpu


# 2. Data
<hr>

load data & apply normalization


In [50]:
from torchvision import transforms, datasets

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

data_path = './MNIST'

training_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)

# 3. Model
<hr>

- design a neural network that consists of three fully connected layers with an activation function of Sigmoid

- the activation function for the output layer is LogSoftmax

In [51]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.Sigmoid(),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.Sigmoid(),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


# 4. Loss Function
<hr>

In [52]:
def loss_function(criterion, y_pred, y):
  return criterion(train_y_pred, train_y)

# Test
model = classification()
criterion = nn.NLLLoss()
train_y_pred = model(training_set.data.float())
train_y = training_set.targets
loss = loss_function(criterion, train_y_pred, train_y)
print(loss)

tensor(2.3767, grad_fn=<NllLossBackward>)


# 5. Optimization
<hr>

In [69]:
def gradient_descent(data_set, batch_size, 
                     learning_rate_value, num_epochs) :

  # classifier
  model = classification().to(device)

  # optimizer
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate_value)

  # loss function
  criterion = nn.NLLLoss()

  # batching
  data_loader =  torch.utils.data.DataLoader(
      data_set,
      batch_size=batch_size,
      shuffle=True)

  # return variables
  epochs, loss_list, acc_list = [], [], []


  # training
  for epoch in range(num_epochs):

    print("epoch : ", epoch, " ================ ")
    loss_sum = 0
    acc_sum = 0
    iteration = 0

    for xs, ts in iter(data_loader):
        
        if len(ts) != batch_size:
          continue
        
        y_pred = model(xs)
        loss = criterion(y_pred, ts)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        iteration = iteration + 1
        loss_sum = loss_sum + float(loss) / batch_size
        zs = y_pred.max(1, keepdim=True)[1]  # get the index of the max logit
        correct = zs.eq(ts.view_as(zs)).sum().item()
        total = int(ts.shape[0])
        acc_sum = acc_sum + correct / total

    if iteration == 0:
      continue

    print("loss : ", loss_sum / iteration)
    print("accuracy : ", acc_sum / iteration)
    loss_list.append(loss_sum / iteration)
    acc_list.append(acc_sum / iteration)
    epochs.append(epoch)

  return epochs, loss_list, acc_list 

In [ ]:
epochs, loss_list, acc_list = gradient_descent(training_set, 32, 0.1, 10)
print(loss_list)

epoch :  0  ================ 
loss :  0.021949435645341873
accuracy :  0.7883833333333333
epoch :  1  ================ 
loss :  0.008129474008424828
accuracy :  0.9235833333333333
epoch :  2  ================ 
loss :  0.005897059512355675
accuracy :  0.9452
epoch :  3  ================ 
loss :  0.004516696946912755
accuracy :  0.9579333333333333
epoch :  4  ================ 
loss :  0.003595477499657621
accuracy :  0.9667
epoch :  5  ================ 
loss :  0.0029407740371612213
accuracy :  0.9733833333333334
epoch :  6  ================ 
loss :  0.0024647668693680317
accuracy :  0.97755
epoch :  7  ================ 
loss :  0.0020884816512931137
accuracy :  0.9812
epoch :  8  ================ 
loss :  0.0017573746757232585
accuracy :  0.9844
epoch :  9  ================ 
